In [ ]:
import pandas as pd

In [ ]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [ ]:
train_data.head()

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
train_data['Survived'].value_counts()

In [ ]:
train_data['Pclass'].value_counts()

In [ ]:
train_data['Embarked'].value_counts()

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self,X):
        return X[self.attribute_names]

In [ ]:
from sklearn.pipeline import Pipeline    #for numerical attributes
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
    ('select_numeric', DataFrameSelector(['Age','SibSp','Parch','Fare'])),
    ('imputer', SimpleImputer(strategy='median')),
])

In [ ]:
num_pipeline.fit_transform(train_data)

In [ ]:
#we also need an imputer for categorical data and a regular SimpleImputer does not work on those 

In [ ]:
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self,X,y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                       index=X.columns)
        return self
    def transform(self,X,y=None):
        return X.fillna(self.most_frequent_)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# pipeline for categorical attributes

In [ ]:
cat_pipeline = Pipeline([
    ('select_cat', DataFrameSelector(['Pclass','Sex','Embarked'])),
    ('imputer', MostFrequentImputer()),
    ('cat_encoder', OneHotEncoder(sparse=False)),
])

In [ ]:
cat_pipeline.fit_transform(train_data)

In [ ]:
#joining the numerical and categorical pipelines

In [ ]:
from sklearn.pipeline import FeatureUnion
preprocess_pipeline= FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

In [ ]:
X_train=preprocess_pipeline.fit_transform(train_data)
X_train

In [ ]:
y_train=train_data['Survived']

In [ ]:
from sklearn.svm import SVC

svm_clf= SVC(gamma='auto')
svm_clf.fit(X_train,y_train)

In [ ]:
#predicting
X_test= preprocess_pipeline.transform(test_data)
y_pred=svm_clf.predict(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score

svm_scores= cross_val_score(svm_clf, X_train,y_train,cv=10)
svm_scores.mean()

### for better accuracy, trying RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_scores = cross_val_score(forest_clf,X_train,y_train, cv=10)
forest_scores.mean()